In [2]:
## imports
import numpy as np
import cv2


## Defining constants

PROTOTXT = "MobileNetSSD_deploy.prototxt"
MODEL = "MobileNetSSD_deploy.caffemodel"
GPU_SUPPORT = 0
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus",  "car", "cat", "chair", "cow", "diningtable", "dog", 
           "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3)) 

net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
if GPU_SUPPORT:
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


    
## importing video from camera or video file


cap = cv2.VideoCapture(0)
count=0
while True:
    ret, frame = cap.read()
    if not ret:
       break
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()
    for i in np.arange(0, detections.shape[2]):
       confidence = detections[0, 0, i, 2]
       if confidence > 0.5:
           idx = int(detections[0, 0, i, 1])
           box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
           (startX, startY, endX, endY) = box.astype("int")
           label = "{}: {:.2f}%".format(CLASSES[idx],confidence*100)
            
            
####  Algorithm to count number of objects in frame and then adding  to total count

           
           if(CLASSES[idx]=="bottle"):
               cv2.rectangle(frame, (startX, startY), (endX, endY),    COLORS[idx], 2)
               y = startY - 15 if startY - 15 > 15 else startY + 15
               cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
           #else:
            # num =0
           count+=i
           cv2.rectangle(frame, (0,0), (440,65), (245,117,16), -1)
           cv2.putText(frame, 'Bottle Count : ', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
           cv2.putText(frame, str(count), 
                        (10,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
           cv2.putText(frame, 'Bottle Count in frame : ', (250,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
           cv2.putText(frame, str(i), 
                        (250,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
###  Functions to show the video and break statement condition

    if(ret==True):
        cv2.imshow('Knee bend video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()